# CategoricalGrouping Notebook - AAAG

In [1]:
# Basics Importation 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

# For Detailed Stats Output
import statsmodels.api as sm
# The linear regression models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
# Instantiating the linear regression models
ols = LinearRegression()
ridge = Ridge()
lasso  = Lasso()
lassocv = LassoCV()
elasticnet = ElasticNet()
elasticnetcv = ElasticNetCV()
# The tree model
from sklearn import tree
# Instantiating the tree model (regression type)
regressor = tree.DecisionTreeRegressor()
# The ensemble model for random forest and bagging
from sklearn import ensemble
# Instantiating the ensemble models
randomForest = ensemble.RandomForestRegressor()
bagging = ensemble.BaggingRegressor()
# Instantiating the boost models
gbm = ensemble.GradientBoostingRegressor()
abr = ensemble.AdaBoostRegressor()
# xg boost
import xgboost as xgb
# lg boost
import lightgbm as lgb
# K mean clustering
from sklearn.cluster import KMeans
kmeans = KMeans()
# The model selection for cross validation, k fold splits, train_test_split, grid search etc. 
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# Some automatic feature selection functions (recursive finding, best feature selection etc.)
import sklearn.feature_selection as fs
# Importing the different error evaluation/metrics
from sklearn import metrics
from sklearn.metrics import silhouette_score
# Importing PCA
from sklearn.decomposition import PCA
# Making it so that we can see all columns of the dataframe
pd.set_option('display.max_columns', None)

# Import data
df=pd.read_csv('standardized_undummified_df_total_garage_car_clean.csv', index_col='Id')
df_test=pd.read_csv('standardized_undummified_df_test_garage_car_clean.csv', index_col='Id')

x=df
y=df.TotalSF
x_test=df_test
y_test=df_test.TotalSF


In [2]:
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold']
ordinal_var_processed=['OverallQual','OverallCond','BedroomAbvGr','TotRmsAbvGrd','Fireplaces',\
             'YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','TotalBaths']
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea',\
          'GrLivArea','GarageYrBlt','TotalSF','CarCapacity']
cont_var_for_tuning=ordinal_var_processed+cont_var_processed

In [3]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [4]:
# TAKES A LONG TIME TO RUN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# TAKES A LONG TIME TO RUN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# K mean clustering performed on categorical variables
# perform for train (non-test) set
nominal_var_cluster_num=[]
nominal_var_cluster_num_test=[]
for cat_var in nominal_var_processed:
    # dummifying the nominal categorical variable
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    print(cat_var)
    # Finding the optimal number of clusters and storing into nominal_var_cluster_num
    kmax = df_for_cluster.shape[1]
    KNumberChoice=range(2, kmax)
    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    # Doing mulitple trials for each category
    cluster_count=[]
    for trial_num in range(0,5):
#         print(trial_num)
        sil = []
        for k in range(2, kmax):
            kmeans = KMeans(n_clusters = k,random_state=trial_num,init='k-means++').fit(df_for_cluster)
            labels = kmeans.labels_
            sil.append(silhouette_score(df_for_cluster, labels, metric = 'euclidean'))
        if np.argmax(sil)<0.2: # defining threshold for 1 cluster
            cluster_count.append(1)
        else:
            cluster_count.append(KNumberChoice[np.argmax(sil)])
    nominal_var_cluster_num.append(max(set(cluster_count), key=cluster_count.count))

    # perform for the test set #######################################       
    dummified_column_test=pd.get_dummies(x_test.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster_test=pd.concat([y_test,dummified_column_test],axis=1,sort='False')
    kmax_test = df_for_cluster_test.shape[1]
    KNumberChoice_test=range(2, kmax_test)
    cluster_count_test=[]
    for trial_num in range(0,5):
#         print(trial_num)
        sil = []
        for k in range(2, kmax_test):
            kmeans_test = KMeans(n_clusters = k,random_state=trial_num,init='k-means++').fit(df_for_cluster_test)
            labels_test = kmeans_test.labels_
            sil.append(silhouette_score(df_for_cluster_test, labels_test, metric = 'euclidean'))
        if np.argmax(sil)<0.2: # defining threshold for 1 cluster
            cluster_count_test.append(1)
        else:
            cluster_count_test.append(KNumberChoice_test[np.argmax(sil)])
    nominal_var_cluster_num_test.append(max(set(cluster_count_test), key=cluster_count_test.count))
print(nominal_var_cluster_num)
print(nominal_var_cluster_num_test)

MSZoning
Street
Alley
LotShape
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
Foundation
BsmtFinType1
Heating
CentralAir
Electrical
Functional
GarageType
GarageFinish
PavedDrive
Fence
MiscFeature
SaleType
SaleCondition
MSSubClass
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MoSold
[5, 1, 1, 4, 1, 5, 1, 24, 8, 1, 5, 7, 4, 1, 15, 16, 5, 6, 7, 1, 1, 4, 1, 6, 4, 1, 3, 1, 8, 5, 16, 1, 1, 1, 1, 1, 1, 12]
[5, 1, 1, 4, 1, 5, 1, 24, 9, 1, 5, 7, 4, 1, 12, 15, 5, 5, 7, 1, 1, 4, 1, 6, 4, 1, 3, 1, 9, 5, 16, 1, 1, 1, 1, 1, 1, 12]


In [5]:
# With the known cluster number for each categorical variable, perform the clustering
# perform for train (non-test) set
for ind,cat_var in enumerate(nominal_var_processed):
    dummified_column=pd.get_dummies(x.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster=pd.concat([y,dummified_column],axis=1,sort='False')
    kmeans=KMeans(n_clusters=nominal_var_cluster_num[ind]).fit(df_for_cluster)
    x.loc[:,cat_var]=kmeans.labels_
    
    # perform for the test set###################
    dummified_column_test=pd.get_dummies(x_test.loc[:,cat_var],prefix=cat_var, prefix_sep='__')
    df_for_cluster_test=pd.concat([y_test,dummified_column_test],axis=1,sort='False')
    kmeans_test=KMeans(n_clusters=nominal_var_cluster_num_test[ind]).fit(df_for_cluster_test)
    x_test.loc[:,cat_var]=kmeans_test.labels_
    
clustered_df=pd.concat([y,x],axis=1,sort='False')    
clustered_df_test=pd.concat([y_test,x_test],axis=1,sort='False')    
print(clustered_df.shape)
print(clustered_df_test.shape)

(2916, 66)
(1459, 65)


In [6]:
# Removing the columns with pure 0's
undummified_clustered_df=clustered_df.loc[:,(clustered_df != 0).any(axis=0)]
undummified_clustered_df.shape

(2916, 49)

In [7]:
cat_cols_to_keep=set(undummified_clustered_df.columns)-set(cont_var_for_tuning)-{'SalePrice'}
# dummifying the clustered_df
undummified_clustered_df_copy=undummified_clustered_df.copy()
for feature in cat_cols_to_keep:
    undummified_clustered_df_copy=dummify_column(undummified_clustered_df_copy,[feature])
dummified_clustered_df=undummified_clustered_df_copy
print(dummified_clustered_df.isnull().sum().sum())
print(dummified_clustered_df.shape)
# dummifying the clustered_df_test
undummified_clustered_df_test_copy=clustered_df_test.copy()
for feature in cat_cols_to_keep:
    undummified_clustered_df_test_copy=dummify_column(undummified_clustered_df_test_copy,[feature])
dummified_clustered_df_test=undummified_clustered_df_test_copy
print(dummified_clustered_df_test.isnull().sum().sum())
print(dummified_clustered_df_test.shape)

1459
(2916, 176)
0
(1459, 189)


In [8]:
dummified_clustered_df_test.head()

,TotalSF,LotFrontage,LotArea,Street,Alley,LandContour,LandSlope,Condition2,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofMatl,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,Heating,HeatingQC,CentralAir,GrLivArea,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscFeature,YrSold,TotalSF,TotalBaths,CarCapacity,LotConfig__1,LotConfig__2,LotConfig__3,LotConfig__4,Exterior1st__1,Exterior1st__2,Exterior1st__3,Exterior1st__4,Exterior1st__5,Exterior1st__6,Exterior1st__7,Exterior1st__8,Exterior1st__9,Exterior1st__10,Exterior1st__11,LotShape__1,LotShape__2,LotShape__3,Neighborhood__1,Neighborhood__2,Neighborhood__3,Neighborhood__4,Neighborhood__5,Neighborhood__6,Neighborhood__7,Neighborhood__8,Neighborhood__9,Neighborhood__10,Neighborhood__11,Neighborhood__12,Neighborhood__13,Neighborhood__14,Neighborhood__15,Neighborhood__16,Neighborhood__17,Neighborhood__18,Neighborhood__19,Neighborhood__20,Neighborhood__21,Neighborhood__22,Neighborhood__23,Foundation__1,Foundation__2,Foundation__3,Foundation__4,MSZoning__1,MSZoning__2,MSZoning__3,MSZoning__4,Exterior2nd__1,Exterior2nd__2,Exterior2nd__3,Exterior2nd__4,Exterior2nd__5,Exterior2nd__6,Exterior2nd__7,Exterior2nd__8,Exterior2nd__9,Exterior2nd__10,Exterior2nd__11,Exterior2nd__12,Exterior2nd__13,Exterior2nd__14,Electrical__1,Electrical__2,Electrical__3,MSSubClass__1,MSSubClass__2,MSSubClass__3,MSSubClass__4,MSSubClass__5,MSSubClass__6,MSSubClass__7,MSSubClass__8,MSSubClass__9,MSSubClass__10,MSSubClass__11,MSSubClass__12,MSSubClass__13,MSSubClass__14,MSSubClass__15,Fence__1,Fence__2,BsmtFinType1__1,BsmtFinType1__2,BsmtFinType1__3,BsmtFinType1__4,BsmtFinType1__5,BsmtFinType1__6,SaleType__1,SaleType__2,SaleType__3,SaleType__4,SaleType__5,SaleType__6,SaleType__7,SaleType__8,BldgType__1,BldgType__2,BldgType__3,BldgType__4,GarageType__1,GarageType__2,GarageType__3,GarageType__4,GarageType__5,GarageFinish__1,GarageFinish__2,GarageFinish__3,MasVnrType__1,MasVnrType__2,MasVnrType__3,MasVnrType__4,MoSold__1,MoSold__2,MoSold__3,MoSold__4,MoSold__5,MoSold__6,MoSold__7,MoSold__8,MoSold__9,MoSold__10,MoSold__11,SaleCondition__1,SaleCondition__2,SaleCondition__3,SaleCondition__4,HouseStyle__1,HouseStyle__2,HouseStyle__3,HouseStyle__4,HouseStyle__5,HouseStyle__6,RoofStyle__1,RoofStyle__2,RoofStyle__3,Condition1__1,Condition1__2,Condition1__3,Condition1__4,Condition1__5,Condition1__6,Condition1__7,Condition1__8
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,-0.952027,0.526150,0.363929,0,0,0,0,0,-0.751101,0.400766,-0.340945,-1.072885,0,-0.563316,-0.678102,-0.223518,-0.813484,0.019105,-0.645887,0,-1.211189,0,-1.215588,-1.029543,-0.772674,-0.918335,0,-0.898055,1.512557e-14,-0.650488,0.203574,0.143211,0,0,0,0,0,0,0,0,1.713905,-0.952027,-1.474576,-0.333310,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1462,0.165162,0.573788,0.897861,0,0,0,0,0,-0.054877,0.400766,-0.439695,-1.214908,0,0.047057,-0.678102,-0.223518,-0.813484,0.019105,-0.645887,0,-1.211189,0,-0.323539,0.175997,0.741518,-0.255371,0,-0.898055,1.512557e-14,-0.767194,0.203574,0.143211,0,0,0,0,0,0,0,0,1.713905,0.165162,-0.873289,-0.879245,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
1463,0.036939,0.240326,0.809646,0,0,0,0,0,-0.751101,-0.497418,0.844059,0.678742,0,-0.563316,-0.

In [9]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Removing sub-categories with low counts and their associated observations (if less than 2% of total observation)
boolmatrix=[]
non_cont_columns=dummified_clustered_df.columns[~dummified_clustered_df.columns.isin(cont_var_for_tuning)]
for cat_feature in non_cont_columns:
    if dummified_clustered_df.loc[:,cat_feature].sum()<0.020*dummified_clustered_df.shape[0]:
        print('yes')
        boolvec=(dummified_clustered_df.loc[:,cat_feature]==1)
        boolmatrix.append(list(boolvec))
   

yes
yes
yes
yes
yes
yes


In [10]:
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
####################### ONLY FOR NON-TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# creating boolean vector to takeout observations that have categorical low count observations
if boolmatrix!=[]:
    reduction_bool_vec=np.any(np.array(boolmatrix).transpose(),axis=1)
    temp_df=dummified_clustered_df[~reduction_bool_vec]
    purged_dummified_clustered_df=temp_df.loc[:,(temp_df != 0).any(axis=0)]
else:
    purged_dummified_clustered_df=dummified_clustered_df
print(purged_dummified_clustered_df.shape)
print(dummified_clustered_df.shape)

(2756, 170)
(2916, 176)


In [11]:
# Removing from the test set columns there were removed from the training set!
cols_to_keep=set(dummified_clustered_df_test.columns)&(set(purged_dummified_clustered_df.columns))


purged_dummified_clustered_df_test=dummified_clustered_df_test.loc[:,list(cols_to_keep)]
purged_dummified_clustered_df=dummified_clustered_df.loc[:,list(cols_to_keep)+['SalePrice']]

# purged_dummified_clustered_df.reset_index(inplace=True)
# purged_dummified_clustered_df.set_index('Id')
# purged_dummified_clustered_df_test.reset_index(inplace=True)
# purged_dummified_clustered_df_test.set_index('Id')
# purged_dummified_clustered_df

# purged_dummified_clustered_df.sample(20)

In [12]:
def undummify(dataframe):
    tot_col=dataframe.columns
    cat_col=list(tot_col[tot_col.str.contains('__')])
    cat_col_split=set(map(lambda x:x.split('__')[0],cat_col))
    cat_dict={}
    for col in cat_col_split:
        sub_df=dataframe[cat_col].loc[:,list(map(lambda x:col in x, dataframe[cat_col].columns))]
        for i in sub_df.columns:
            label_num=int(i.split('__')[1])
            sub_df.loc[:,i]=np.array(sub_df.loc[:,i])*label_num
        cat_dict[col]=sub_df.sum(axis=1)+1
    df1=dataframe.drop(cat_col,axis=1)
    df2=pd.DataFrame(cat_dict)
    return pd.concat([df1,df2],axis=1)

In [13]:
purged_undummified_clustered_df=undummify(purged_dummified_clustered_df)
purged_undummified_clustered_df_test=undummify(purged_dummified_clustered_df_test)

purged_dummified_clustered_df.to_csv('purged_dummified_clustered_df_total_garage_car_clean.csv')
purged_dummified_clustered_df_test.to_csv('purged_dummified_clustered_df_test_garage_car_clean.csv')

purged_undummified_clustered_df.to_csv('purged_undummified_clustered_df_total_garage_car_clean.csv')
purged_undummified_clustered_df_test.to_csv('purged_undummified_clustered_df_test_garage_car_clean.csv')

In [14]:
############# Reducing Continuous/Ordinal Only ################
cont_df=purged_dummified_clustered_df.loc[:,cont_var_for_tuning]
pca = PCA(n_components = 0.90)
pca.fit(cont_df)
reduced_cont_df = pd.DataFrame(pca.transform(cont_df))
# Performing for the test set
cont_df_test=purged_dummified_clustered_df_test.loc[:,cont_var_for_tuning]
pca_test = PCA(n_components = 0.90)
pca_test.fit(cont_df_test)
reduced_cont_df_test = pd.DataFrame(pca_test.transform(cont_df_test))

In [15]:
categorical_purged_dummified_clustered_df=purged_dummified_clustered_df.drop(cont_var_for_tuning,axis=1)
categorical_purged_dummified_clustered_df_test=purged_dummified_clustered_df_test.drop(cont_var_for_tuning,axis=1)

PCA_purged_dummified_clustered_df=pd.concat([categorical_purged_dummified_clustered_df.reset_index(),reduced_cont_df],axis=1)
PCA_purged_dummified_clustered_df_test=pd.concat([categorical_purged_dummified_clustered_df_test.reset_index(),reduced_cont_df_test],axis=1)

In [16]:
PCA_purged_dummified_clustered_df.set_index('Id',inplace=True)
PCA_purged_dummified_clustered_df_test.set_index('Id',inplace=True)
PCA_purged_dummified_clustered_df.to_csv('PCA_purged_dummified_clustered_df.csv')
PCA_purged_dummified_clustered_df_test.to_csv('PCA_dummified_clustered_df_test.csv')

# My Attempt to improve on k mean clustering 
# A Working progress

In [17]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# from scipy.spatial.distance import cdist
# import numpy as np
# import matplotlib.pyplot as plt

# # k means determine k
# distortions = []
# kmax = df_for_cluster.shape[1]
# K = range(1,kmax)
# for k in K:
#     kmeanModel = KMeans(n_clusters=k).fit(df_for_cluster)
#     kmeanModel.fit(df_for_cluster)
#     distortions.append(sum(np.min(cdist(df_for_cluster, kmeanModel.cluster_centers_, 'euclidean')+0*(k), axis=1)) / df_for_cluster.shape[0])

# # Plot the elbow
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

In [18]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# from scipy.spatial.distance import cdist
# import numpy as np
# import matplotlib.pyplot as plt

# # k means determine k
# K = range(1,df_for_cluster.shape[1])
# penalty_term=np.linspace(0.005,0.01,101)
# distortion_matrix = np.zeros([len(K),len(penalty_term)])
# for i,k in enumerate(K):
#     for j,pen_weight in enumerate(penalty_term):
#         #print(i,j)
#         kmeanModel = KMeans(n_clusters=k).fit(df_for_cluster)
#         kmeanModel.fit(df_for_cluster)
#         distortion_matrix[i,j]=sum(np.min(cdist(df_for_cluster, kmeanModel.cluster_centers_, 'euclidean')+pen_weight*(k), axis=1)) / df_for_cluster.shape[0]
# # The minimum index
# np.unravel_index(np.argmin(distortion_matrix, axis=None), distortion_matrix.shape)
# # Plot the elbow
# plt.pcolor(distortion_matrix)
# plt.colorbar();
# plt.xlabel('Penalization');
# plt.ylabel('k clusters');